# Capstone 1 : Analysis, Part 1

In this section, we will apply ML techniques to the transcript and the features we extracted to predict the ratings

In [253]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns



In [254]:
# Functon for text normalization
def text_preprocess(text):
    # Strip leading and lagging whitespace
    text = text.strip()
    
    # Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Convert all text to lower case
    text.lower()
    
    # Take raw text and remove all audience reactions
    text = re.sub('\((.*?)\)', '', text)
    
    # Expand Contactions
    # Specific
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    
    # Remove Punctuation
    pattern = r'[^a-zA-z\s]'
    text = re.sub(pattern, ' ', text)
    
    return text


#     # Remove punctuation
#     normalized_text.translate( str.maketrans('','', string.punctuation))
#     # Word tokenization
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(normalized_text)
#     result = [i for i in tokens if not i in stop_words]
#     stemmer = PorterStemmer()
#     stemmed_result = []
#     for word in result:
#         stemmed_result.append(stemmer.stem(word))
#     return stemmed_result

In [255]:
# Read data after statistical analysis
df_clean = pd.read_csv(r'After_StatisticalAnalysis.csv', index_col = 0)
df_clean.index.name = "index"

In [256]:
df_clean.head(2)

,description,duration,event,main_speaker,speaker_occupation,tags,title,transcript,film_datestamp,pub_datestamp,...,Inspiring,event_type,Max_rating,ratings_total,word_per_min,tag_technology,tag_science,tag_global issues,tag_culture,tag_design
index,,,,,,,,,,,,,,,,,,,,,
0,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,Ken Robinson,Author/educator,"['tag_children', 'tag_creativity', 'tag_cultur...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,2006-02-25 00:00:00,2006-06-27 00:11:00,...,0.398474,TED,Inspiring,89411,163,False,False,False,True,False
1,With the same humor and humanity he exuded in ...,977,TED2006,Al Gore,Climate advocate,"['tag_alternative energy', 'tag_cars', 'tag_cl...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",2006-02-25 00:00:00,2006-06-27 00:11:00,...,0.241489,TED,Inspiring,2820,127,True,True,True,True,False


In [257]:
df_clean['clean_transcript'] = df_clean.transcript.apply(text_preprocess)

In [279]:
# Create class mapping
class_dict = ({'Inspiring': 1, 'Funny' : 2, 'Informative' : 3,'Ingenious' : 4, 
                 'Beautiful' : 5, 'Confusing' :6, 'Courageous':7, 'Fascinating' : 8, 
                'OK' :9 , 'Unconvincing' : 10})
df_clean['MaxRating_Class'] =  df_clean.Max_rating.map(class_dict)

In [280]:
df_clean.head(2)

,description,duration,event,main_speaker,speaker_occupation,tags,title,transcript,film_datestamp,pub_datestamp,...,Max_rating,ratings_total,word_per_min,tag_technology,tag_science,tag_global issues,tag_culture,tag_design,clean_transcript,MaxRating_Class
index,,,,,,,,,,,,,,,,,,,,,
0,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,Ken Robinson,Author/educator,"['tag_children', 'tag_creativity', 'tag_cultur...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,2006-02-25 00:00:00,2006-06-27 00:11:00,...,Inspiring,89411,163,False,False,False,True,False,Good morning How are you It is been great ha...,1
1,With the same humor and humanity he exuded in ...,977,TED2006,Al Gore,Climate advocate,"['tag_alternative energy', 'tag_cars', 'tag_cl...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",2006-02-25 00:00:00,2006-06-27 00:11:00,...,Inspiring,2820,127,True,True,True,True,False,Thank you so much Chris And it is truly a gr...,1


In [281]:
df_clean.columns

Index(['description', 'duration', 'event', 'main_speaker',
       'speaker_occupation', 'tags', 'title', 'transcript', 'film_datestamp',
       'pub_datestamp', 'sentence_count', 'word_count', 'applause', 'laughter',
       'music', 'cheering', 'sighs', 'singing', 'video', 'audio', 'Funny',
       'Informative', 'Inspiring', 'event_type', 'Max_rating', 'ratings_total',
       'word_per_min', 'tag_technology', 'tag_science', 'tag_global issues',
       'tag_culture', 'tag_design', 'clean_transcript', 'MaxRating_Class'],
      dtype='object')

## Find the best predictors(words) for each ratings

### Define Factors and targets for NB model

In [282]:
X_metadata = df_clean[['description', 'duration', 'event', 'main_speaker',
       'speaker_occupation', 'title', 'transcript', 'sentence_count', 'word_count', 'applause', 'laughter',
       'music', 'cheering', 'sighs', 'singing', 'video', 'audio', 'event_type', 'ratings_total',
       'word_per_min', 'tag_technology', 'tag_science', 'tag_global issues',
       'tag_culture', 'tag_design']]
# X_transcript = df_clean[['transcript']]

y_Funny = df_clean[['Funny']]
y_Informative = df_clean[['Informative']]
y_Inspiring = df_clean[['Inspiring']]
y_MaxRating = df_clean[['MaxRating_Class']]

In [283]:
# X_transcript.head()

In [299]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words = {'english'}, min_df = 0.02, max_df = 0.95)
vectorizer.fit(df_clean.clean_transcript)
X_transcript = vectorizer.transform(df_clean.clean_transcript)
# X_transcript = X_transcript.todense()

In [300]:
type(X_transcript)

scipy.sparse.csr.csr_matrix

In [301]:
X_transcript.shape

(2314, 4336)

In [302]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.95, max_features=None, min_df=0.02,
                ngram_range=(1, 1), preprocessor=None, stop_words={'english'},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [303]:
# vectorizer.vocabulary_

### Train the Naive Bayes Model

Pre-processing
Before training a naive Bayes Classifier to get best predictor words, we will perform text normalization on the transcripts. We will do the following
1. Remove accented characters and special characters
2. Remove comments and audience reactions in parenthesis
2. Make all text lower cased
3. Expand contractions
 

In [304]:
import spacy
import nltk
import re
import unicodedata

In [312]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import balanced_accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_transcript, (y_MaxRating), test_size = 0.2)
nb = MultinomialNB(alpha=100)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

# Calculate accuracy score using balanced accuracy scores
print ('Balanced accuracy score (training)is ', balanced_accuracy_score(nb.predict(X_train), y_train))
print ('Balanced accuracy score (test)is ', balanced_accuracy_score(nb.predict(X_test), y_test))

# print ('Accuracy of model on training set is ', nb.score(X_train, y_train))
# print ('Accuracy of model on test set is ', nb.score(X_test, y_test))


Balanced accuracy score (training)is  0.8318196891413412
Balanced accuracy score (test)is  0.7852494577006508


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


This model has large difference between training and test because the classes are severely unbalanced. However, we will not optimize it further at this point. Instead we will continue to use it for predicting words indicative of each rating


In [314]:
# Train model with entire dataset
nb = MultinomialNB(alpha = 100)
nb.fit(X_transcript, y_MaxRating)
y_pred = nb.predict(X_test)

print ('Balanced accuracy score (test)is ', balanced_accuracy_score(nb.predict(X_transcript), y_MaxRating))

Balanced accuracy score (test)is  0.7651236668935785


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


### Find words indicative of each rating

In [315]:
words = np.array(vectorizer.get_feature_names())

In [334]:
words.shape

(4336,)

In [325]:
x = np.eye(X_transcript.shape[1])


In [327]:
class_dict

{'Inspiring': 1,
 'Funny': 2,
 'Informative': 3,
 'Ingenious': 4,
 'Beautiful': 5,
 'Confusing': 6,
 'Courageous': 7,
 'Fascinating': 8,
 'OK': 9,
 'Unconvincing': 10}

Now, for each rating, w ecan find the words with highest probability and lowest probability for a particular rating. 
So, we will look at best and worst predictors of three ratings : Inspiring, Funny and Informative

In [347]:
# For 'Inspiring'
probs_inspiring = nb.predict_log_proba(x)[:, 0]
ind = np.argsort(probs_inspiring)

good_words = words[ind[:15]]
bad_words = words[ind[-15:]]

good_prob = probs_inspiring[ind[:15]]
bad_prob = probs_inspiring[ind[-15:]]

print("Good words\t     P(Inspiring | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
# print("Bad words\t     P(Inspiring | word)")
# for w, p in zip(bad_words, bad_prob):
#     print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

Good words	     P(Inspiring | word)
             neurons 0.75
           telescope 0.73
                 dna 0.72
               genes 0.71
             planets 0.71
              genome 0.70
               males 0.70
             quantum 0.70
           particles 0.69
            magnetic 0.69
              cortex 0.69
            organism 0.69
                 tag 0.69
             imaging 0.69
             mammals 0.68


In [346]:
# For 'Funny'
probs_funny = nb.predict_log_proba(x)[:, 1]
ind = np.argsort(probs_funny)

good_words = words[ind[:15]]
bad_words = words[ind[-15:]]

good_prob = probs_funny[ind[:15]]
bad_prob = probs_funny[ind[-15:]]

print("Good words\t     P(Funny | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

# print("Bad words\t     P(Funny | word)")
# for w, p in zip(bad_words, bad_prob):
#     print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

Good words	     P(Funny | word)
                 our 0.99
               world 0.99
               their 0.99
                  us 0.98
                need 0.98
              people 0.98
               those 0.98
             percent 0.98
               years 0.98
               these 0.98
              change 0.98
                most 0.98
                many 0.98
                been 0.98
                more 0.98
Bad words	     P(Funny | word)
               honey 0.90
         advertising 0.90
              emails 0.90
                joke 0.90
              dating 0.90
                 dan 0.90
                 huh 0.89
             cartoon 0.89
               males 0.89
               hello 0.89
               laugh 0.89
                  hi 0.89
            laughing 0.88
               humor 0.87
                flag 0.86


In [349]:
# For 'Informative'
probs_informative = nb.predict_log_proba(x)[:, 2]
ind = np.argsort(probs_informative)

good_words = words[ind[:15]]
bad_words = words[ind[-15:]]

good_prob = probs_informative[ind[:15]]
bad_prob = probs_informative[ind[-15:]]

print("Good words\t     P(Informative | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Bad words\t     P(Informative | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

Good words	     P(Informative | word)
                  my 0.92
                 she 0.92
              school 0.92
                kids 0.91
                  me 0.90
               girls 0.90
                 her 0.90
                said 0.90
                feel 0.90
                 him 0.90
          compassion 0.90
              myself 0.90
                felt 0.90
                 god 0.89
                love 0.89
Bad words	     P(Informative | word)
               drugs 0.69
             disease 0.69
             privacy 0.69
             planets 0.69
               virus 0.68
             genetic 0.68
              genome 0.67
                 flu 0.67
                data 0.67
             neurons 0.66
                gene 0.66
               sleep 0.65
               brain 0.65
                 dna 0.64
               genes 0.62
